In [ ]:
!pip install sentence-transformers==2.7.0 datasets==2.19.0 huggingface_hub==0.23.0 faiss-cpu==1.8.0 -qqq

## 예제 11.1 사전 학습된 언어 모델을 불러와 문장 임베딩 모델 만들기

In [ ]:
from sentence_transformers import SentenceTransformer, models
transformer_model = models.Transformer('klue/roberta-base')

pooling_layer = models.Pooling(
    transformer_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)
embedding_model = SentenceTransformer(modules=[transformer_model, pooling_layer])

## 예제 11.2 실습 데이터셋 다운로드 및 확인

In [ ]:
from datasets import load_dataset
klue_sts_train = load_dataset('klue', 'sts', split='train')
klue_sts_test = load_dataset('klue', 'sts', split='validation')
klue_sts_train[0]

# {'guid': 'klue-sts-v1_train_00000',
#  'source': 'airbnb-rtt',
#  'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
#  'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
#  'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

## 예제 11.3 학습 데이터에서 검증 데이터셋 분리하기

In [ ]:
# 학습 데이터셋의 10%를 검증 데이터셋으로 구성한다.
klue_sts_train = klue_sts_train.train_test_split(test_size=0.1, seed=42)
klue_sts_train, klue_sts_eval = klue_sts_train['train'], klue_sts_train['test']

## 예제 11.4 label 정규화하기

In [ ]:
from sentence_transformers import InputExample
# 유사도 점수를 0~1 사이로 정규화 하고 InputExample 객체에 담는다.
def prepare_sts_examples(dataset):
    examples = []
    for data in dataset:
        examples.append(
            InputExample(
                texts=[data['sentence1'], data['sentence2']],
                label=data['labels']['label'] / 5.0)
            )
    return examples

train_examples = prepare_sts_examples(klue_sts_train)
eval_examples = prepare_sts_examples(klue_sts_eval)
test_examples = prepare_sts_examples(klue_sts_test)

## 예제 11.5 학습에 사용할 배치 데이터셋 만들기

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

## 예제 11.6 검증을 위한 평가 객체 준비

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

eval_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(eval_examples)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_examples)

## 예제 11.7 언어 모델을 그대로 활용할 경우 문장 임베딩 모델의 성능

In [ ]:
test_evaluator(embedding_model)
# 0.36460670798564826

## 예제 11.8 임베딩 모델 학습

In [ ]:
from sentence_transformers import losses

num_epochs = 4
model_name = 'klue/roberta-base'
model_save_path = 'output/training_sts_' + model_name.replace("/", "-")
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# 임베딩 모델 학습
embedding_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=eval_evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=100,
    output_path=model_save_path
)

## 예제 11.9 학습한 임베딩 모델의 성능 평가

In [ ]:
trained_embedding_model = SentenceTransformer(model_save_path)
test_evaluator(trained_embedding_model)
# 0.8965595666246748

## 예제 11.10 허깅페이스 허브에 모델 저장

In [ ]:
from huggingface_hub import login
from huggingface_hub import HfApi

login(token='허깅페이스 허브 토큰 입력')
api = HfApi()
repo_id="klue-roberta-base-klue-sts"
api.create_repo(repo_id=repo_id)

api.upload_folder(
    folder_path=model_save_path,
    repo_id=f"본인의 허깅페이스 아이디 입력/{repo_id}",
    repo_type="model",
)

## 예제 11.11 실습 데이터를 내려받고 예시 데이터 확인

In [ ]:
from datasets import load_dataset
klue_mrc_train = load_dataset('klue', 'mrc', split='train')
klue_mrc_train[0]
# {'title': '제주도 장마 시작 … 중부는 이달 말부터',
#  'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
#  'news_category': '종합',
#  'source': 'hankyung',
#  'guid': 'klue-mrc-v1_train_12759',
#  'is_impossible': False,
#  'question_type': 1,
#  'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
#  'answers': {'answer_start': [478, 478], 'text': ['한 달가량', '한 달']}}

## 예제 11.12 기본 임베딩 모델 불러오기

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('shangrilar/klue-roberta-base-klue-sts')

## 예제 11.13 데이터 전처리

In [ ]:
from datasets import load_dataset
klue_mrc_train = load_dataset('klue', 'mrc', split='train')
klue_mrc_test = load_dataset('klue', 'mrc', split='validation')

df_train = klue_mrc_train.to_pandas()
df_test = klue_mrc_test.to_pandas()

df_train = df_train[['title', 'question', 'context']]
df_test = df_test[['title', 'question', 'context']]

## 예제 11.14 질문과 관련이 없는 기사를 irrelevant_context 컬럼에 추가

In [ ]:
def add_ir_context(df):
  irrelevant_contexts = []
  for idx, row in df.iterrows():
    title = row['title']
    irrelevant_contexts.append(df.query(f"title != '{title}'").sample(n=1)['context'].values[0])
  df['irrelevant_context'] = irrelevant_contexts
  return df

df_train_ir = add_ir_context(df_train)
df_test_ir = add_ir_context(df_test)

## 예제 11.15 성능 평가에 사용할 데이터 생성

In [ ]:
from sentence_transformers import InputExample

examples = []
for idx, row in df_test_ir[:100].iterrows():
  examples.append(
      InputExample(texts=[row['question'], row['context']], label=1)
  )
  examples.append(
      InputExample(texts=[row['question'], row['irrelevant_context']], label=0)
  )

## 예제 11.16 기본 임베딩 모델의 성능 평가 결과

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    examples
)
evaluator(sentence_model)
# 0.8151553052035344

## 예제 11.17 긍정 데이터만으로 학습 데이터 구성

In [ ]:
train_samples = []
for idx, row in df_train_ir.iterrows():
    train_samples.append(InputExample(
        texts=[row['question'], row['context']]
    ))

## 예제 11.18 중복 학습 데이터 제거

In [ ]:
from sentence_transformers import datasets

batch_size = 16

loader = datasets.NoDuplicatesDataLoader(
    train_samples, batch_size=batch_size)

## 예제 11.19 MNR 손실 함수 불러오기

In [ ]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(sentence_model)

## 예제 11.20 MRC 데이터셋으로 미세 조정

In [ ]:
epochs = 1
save_path = './klue_mrc_mnr'

sentence_model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=100,
    output_path=save_path,
    show_progress_bar=True
)

## 예제 11.21 미세 조정한 모델 성능 평가

In [ ]:
evaluator(sentence_model)
# 0.8600968992433692

## 예제 11.22 허깅페이스 허브에 미세 조정한 모델 업로드

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
repo_id = "klue-roberta-base-klue-sts-mrc"
api.create_repo(repo_id=repo_id)

api.upload_folder(
    folder_path=save_path,
    repo_id=f"본인의 아이디 입력/{repo_id}",
    repo_type="model",
)

## 예제 11.23 교차 인코더로 사용할 사전 학습 모델 불러오기

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder
cross_model = CrossEncoder('klue/roberta-small', num_labels=1)

## 예제 11.24 미세 조정하지 않은 교차 인코더의 성능 평가 결과

In [ ]:
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
ce_evaluator = CECorrelationEvaluator.from_input_examples(examples)
ce_evaluator(cross_model)
# 0.003316821814673943

## 예제 11.25 교차 인코더 학습 데이터셋 준비

In [ ]:
train_samples = []
for idx, row in df_train_ir.iterrows():
    train_samples.append(InputExample(
        texts=[row['question'], row['context']], label=1
    ))
    train_samples.append(InputExample(
        texts=[row['question'], row['irrelevant_context']], label=0
    ))

## 예제 11.26 교차 인코더 학습 수행

In [ ]:
train_batch_size = 16
num_epochs = 1
model_save_path = 'output/training_mrc'

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

cross_model.fit(
    train_dataloader=train_dataloader,
    epochs=num_epochs,
    warmup_steps=100,
    output_path=model_save_path
)

## 예제 11.27 학습한 교차 인코더 평가 결과

In [ ]:
ce_evaluator(cross_model)
# 0.8650250798639563

## 예제 11.28 학습을 마친 교차 인코더를 허깅페이스 허브에 업로드

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
repo_id = "klue-roberta-small-cross-encoder"
api.create_repo(repo_id=repo_id)

api.upload_folder(
    folder_path=model_save_path,
    repo_id=f"본인의 아이디 입력/{repo_id}",
    repo_type="model",
)

## 예제 11.29 평가를 위한 데이터셋을 불러와 1,000개만 선별

In [ ]:
from datasets import load_dataset
klue_mrc_test = load_dataset('klue', 'mrc', split='validation')
klue_mrc_test = klue_mrc_test.train_test_split(test_size=1000, seed=42)['test']

## 예제 11.30 임베딩을 저장하고 검색하는 함수 구현

In [ ]:
import faiss
def make_embedding_index(sentence_model, corpus):
	embeddings = sentence_model.encode(corpus)
	index = faiss.IndexFlatL2(embeddings.shape[1])
	index.add(embeddings)
	return index

def find_embedding_top_k(query, sentence_model, index, k):
	embedding = sentence_model.encode([query])
	distances, indices = index.search(embedding, k)
	return indices

## 예제 11.31 교차 인코더를 활용한 순위 재정렬 함수 정의

In [ ]:
def make_question_context_pairs(question_idx, indices):
  return [[klue_mrc_test['question'][question_idx], klue_mrc_test['context'][idx]] for idx in indices]

def rerank_top_k(cross_model, question_idx, indices, k):
  input_examples = make_question_context_pairs(question_idx, indices)
  relevance_scores = cross_model.predict(input_examples)
  reranked_indices = indices[np.argsort(relevance_scores)[::-1]]
  return reranked_indices

## 예제 11.32 성능 지표(히트율)와 평가에 걸린 시간을 반환하는 함수 정의

In [ ]:
import time
def evaluate_hit_rate(datasets, embedding_model, index, k=10):
  start_time = time.time()
  predictions = []
  for question in datasets['question']:
    predictions.append(find_embedding_top_k(question, embedding_model, index, k)[0])
  total_prediction_count = len(predictions)
  hit_count = 0
  questions = datasets['question']
  contexts = datasets['context']
  for idx, prediction in enumerate(predictions):
    for pred in prediction:
      if contexts[pred] == contexts[idx]:
        hit_count += 1
        break
  end_time = time.time()
  return hit_count / total_prediction_count, end_time - start_time

## 예제 11.33 기본 임베딩 모델 평가

In [ ]:
from sentence_transformers import SentenceTransformer
base_embedding_model = SentenceTransformer('shangrilar/klue-roberta-base-klue-sts')
base_index = make_embedding_index(base_embedding_model, klue_mrc_test['context'])
evaluate_hit_rate(klue_mrc_test, base_embedding_model, base_index, 10)
# (0.88, 13.216430425643921)

## 예제 11.34 미세 조정한 임베딩 모델 평가

In [ ]:
finetuned_embedding_model = SentenceTransformer('shangrilar/klue-roberta-base-klue-sts-mrc')
finetuned_index = make_embedding_index(finetuned_embedding_model, klue_mrc_test['context'])
evaluate_hit_rate(klue_mrc_test, finetuned_embedding_model, finetuned_index, 10)
# (0.946, 14.309881687164307)

## 예제 11.35 순위 재정렬을 포함한 평가 함수

In [ ]:
import time
import numpy as np
from tqdm.auto import tqdm

def evaluate_hit_rate_with_rerank(datasets, embedding_model, cross_model, index, bi_k=30, cross_k=10):
  start_time = time.time()
  predictions = []
  for question_idx, question in enumerate(tqdm(datasets['question'])):
    indices = find_embedding_top_k(question, embedding_model, index, bi_k)[0]
    predictions.append(rerank_top_k(cross_model, question_idx, indices, k=cross_k))
  total_prediction_count = len(predictions)
  hit_count = 0
  questions = datasets['question']
  contexts = datasets['context']
  for idx, prediction in enumerate(predictions):
    for pred in prediction:
      if contexts[pred] == contexts[idx]:
        hit_count += 1
        break
  end_time = time.time()
  return hit_count / total_prediction_count, end_time - start_time, predictions

## 예제 11.36 임베딩 모델과 교차 인코드를 조합해 성능 평가

In [ ]:
hit_rate, cosumed_time, predictions = evaluate_hit_rate_with_rerank(klue_mrc_test, finetuned_embedding_model, cross_model, finetuned_index, bi_k=30, cross_k=10)
hit_rate, cosumed_time
# (0.973, 1103.055629491806)